<h1>Yahoo Finance Ticker Statistics Page Scraper</h1>
<h3>This script takes a list of tickers and scraps its info from the yahoo finance statistics page.</h3>

<a href = "https://finance.yahoo.com/quote/TSLA/key-statistics?p=TSLA"><img src="sample.JPG"></a>

In [139]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import time
from tabulate import tabulate

<h2>Reading portfolio tickers from csv file</h2>

In [73]:
input_tickers = pd.read_csv('portfolio_tickers.csv')
tickers = input_tickers['Tickers'].tolist()

In [3]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
soups=[]
n = 1

for ticker in tickers:
    url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
    resp = requests.get(url, headers = headers)
    print(f'{ticker} status - {resp.status_code}, {n}/{len(tickers)}')
    #time.sleep(1)
    soup = BeautifulSoup(resp.text, "html.parser")
    soups.append(soup)
    n+=1
print('Extraction complete!')

TSLA 200
TTD 200
NVDA 200
GOOG 200
CRWD 200
AMZN 200
IDXX 200
MELI 200
ABNB 200
ZS 200
ASML 200
SE 200
AXON 200
TEAM 200
LULU 200
SNOW 200
NET 200
HUBS 200
DDOG 200
DOCN 200
U 200
SEMR 200
LMND 200
ZM 200
CDNS 200
PINS 200
MDB 200
MRNA 200
ROKU 200
GLBE 200
ASAN 200
RBLX 200
UPST 200
TWLO 200
CRSP 200
APPN 200
NNOX 200
SHOP 200
Extraction complete!


<h2>Getting the headers and cleaning dates/annotations</h2>

In [4]:
tds = soup.find_all("td")

n= 0
titles= []

for td in tds:
    n+=1
    if n%2 == 0:
        continue
    else:
        text = td.text.strip()
        titles.append(text)
    
all_stats_df = pd.DataFrame({"Metrics":titles})
all_stats_df.head()

,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)


In [5]:
all_stats_df['Metrics'].replace(regex={r'[0-9]$': ''}, inplace = True) 
#removes the annotations appearing at the end of rows

display(all_stats_df.iloc[23:28])
all_stats_df.iloc[23:28,0].replace(regex={r'(\(.+\))': ''}, inplace = True) 
#remove the dates under rows 23-27

all_stats_df

,Metrics
23,"Shares Short (Nov 29, 2022)"
24,"Short Ratio (Nov 29, 2022)"
25,"Short % of Float (Nov 29, 2022)"
26,"Short % of Shares Outstanding (Nov 29, 2022)"
27,"Shares Short (prior month Oct 30, 2022)"


,Metrics
0,Market Cap (intraday)
1,Enterprise Value
2,Trailing P/E
3,Forward P/E
4,PEG Ratio (5 yr expected)
5,Price/Sales (ttm)
6,Price/Book (mrq)
7,Enterprise Value/Revenue
8,Enterprise Value/EBITDA
9,Beta (5Y Monthly)


<h2>Retrieving the stats for each ticker</h2>

In [6]:
for soup, ticker in zip(soups, tickers):
    tds = soup.find_all("td")
    n= 0
    stats= []

    for td in tds:
        n+=1
        if n%2 != 0:
            continue
        else:
            text = td.text.strip()
            stats.append(text)
    all_stats_df[f'{ticker}'] = stats
    
all_stats_df

,Metrics,TSLA,TTD,NVDA,GOOG,CRWD,AMZN,IDXX,MELI,ABNB,...,ROKU,GLBE,ASAN,RBLX,UPST,TWLO,CRSP,APPN,NNOX,SHOP
0,Market Cap (intraday),395.82B,22.81B,405.92B,1.14T,25.16B,854.80B,34.20B,45.27B,53.74B,...,5.98B,3.15B,2.84B,16.57B,1.18B,8.29B,3.61B,2.49B,436.44M,46.04B
1,Enterprise Value,380.59B,21.75B,404.53B,1.05T,23.46B,924.39B,35.61B,47.25B,46.47B,...,4.67B,2.98B,2.55B,15.03B,N/A,5.34B,1.95B,2.46B,362.20M,42.39B
2,Trailing P/E,38.77,N/A,70.22,17.55,N/A,76.14,52.54,167.93,35.22,...,N/A,N/A,N/A,N/A,N/A,N/A,124.34,N/A,N/A,N/A
3,Forward P/E,24.69,40.16,38.31,16.50,51.02,41.32,48.31,64.10,27.70,...,N/A,N/A,N/A,N/A,N/A,344.83,N/A,N/A,N/A,1.43k
4,PEG Ratio (5 yr expected),1.10,2.33,3.92,1.24,1.25,4.66,6.92,0.91,N/A,...,N/A,N/A,N/A,N/A,N/A,44.96,N/A,N/A,N/A,N/A
5,Price/Sales (ttm),5.78,15.39,14.58,4.16,12.23,1.70,10.53,4.75,7.08,...,1.90,8.86,5.21,7.36,1.25,2.24,281.78,5.53,N/A,8.74
6,Price/Book (mrq),9.93,11.91,19.01,4.50,19.15,6.22,71.85,27.76,9.70,...,2.15,3.46,7.39,39.11,1.64,0.79,1.85,14.40,1.76,5.30
7,Enterprise Value/Revenue,5.08,14.67,14.16,3.73,11.53,1.84,10.66,4.89,5.79,...,1.49,8.46,5.02,6.79,N/A,1.46,152.92,5.49,45.59,8.08
8,Enterprise Value/EBITDA,23.48,572.96,54.34,11.17,-711.89,18.23,36.21,44.52,25.29,...,-35.90,-42.74,-6.65,-23.36,N/A,-6.48,-2.95,-18.12,-4.85,-12.42
9,Beta (5Y Monthly),1.91,1.74,1.75,1.06,1.06,1.19,1.18,1.52,0.93,...,1.52,N/A,1.04,N/A,1.06,1.37,1.70,1.55,1.58,1.90


<h2>Renaming the columns to indicate units clearly</h2>

In [86]:
col_list = all_stats_df['Metrics']
col_list[0] = 'Market Cap (B)'
col_list[1] = 'Enterprise Value (B)'
col_list[10] = '52 Week Change (%)'
col_list[11] = 'S&P500 52-Week Change (%)'
col_list[16] = 'Avg Vol 3 month (M)'
col_list[17] = 'Avg Vol 10 day (M)'
col_list[18] = 'Shares Outstanding (M)'
col_list[19] = 'Implied Shares Outstanding (M)'
col_list[20] = 'Float'
col_list[23] = 'Shares Short (M)'
col_list[27] = 'Shares Short Prior Month (M)'
col_list[29] = 'Forward Annual Dividend Yield (%)'
col_list[33] = 'Trailing Annual Dividend Yield (%)'
col_list[33] = 'Payout Ratio (%)'
col_list[36] = 'Last Split Factor (x:1)'
col_list[40] = 'Profit Margin (%)'
col_list[41] = 'Operating Margin (ttm) (%)'
col_list[42] = 'Return on Assets (ttm) (%)'
col_list[43] = 'Return on Equity (ttm) (%)'
col_list[44] = 'Revenue (ttm) (B)'
col_list[46] = 'Quarterly Revenue Growth (yoy) (%)'
col_list[47] = 'Gross Profit (ttm) (B)'
col_list[48] = 'EBITDA (B)'
col_list[49] = 'Net Income Avi to Common (ttm) (B)'
col_list[51] = 'Quarterly Earnings Growth (yoy) (%)'
col_list[52] = 'Total Cash (mrq) (B)'
col_list[53] = 'Total Debt (mrq) (B)'
col_list[58] = 'Operating Cash Flow (ttm) (B)'
col_list[59] = 'Levered Free Cash Flow (ttm) (B)'

<h2>Transposing and cleaning the data to numericals</h2>

In [150]:
t_all_stats_df = all_stats_df.transpose()
t_all_stats_df.columns = col_list
t_all_stats_df.drop(index='Metrics', inplace= True)
t_all_stats_df.reset_index(inplace=True)
t_all_stats_df.rename({'index' : 'Ticker'}, axis = 1, inplace=True)

t_all_stats_df.head()

Metrics,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy) (%),Total Cash (mrq) (B),Total Debt (mrq) (B),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B)
0,TSLA,395.82B,380.59B,38.77,24.69,1.10,5.78,9.93,5.08,23.48,...,3.23,103.50%,21.11B,6.68,5.87B,14.28,1.46,12.62,16.03B,6.55B
1,TTD,22.81B,21.75B,N/A,40.16,2.33,15.39,11.91,14.67,572.96,...,0.08,-73.30%,1.32B,2.7,266.75M,13.94,1.91,3.91,538.65M,540.31M
2,NVDA,405.92B,404.53B,70.22,38.31,3.92,14.58,19.01,14.16,54.34,...,3.73,-72.40%,13.14B,5.34,11.9B,55.76,3.39,8.65,6.43B,5.15B
3,GOOG,1.14T,1.05T,17.55,16.50,1.24,4.16,4.50,3.73,11.17,...,4.92,-26.50%,116.26B,8.98,29.34B,11.57,2.52,19.55,92.81B,48.26B
4,CRWD,25.16B,23.46B,N/A,51.02,1.25,12.23,19.15,11.53,-711.89,...,-0.79,N/A,2.47B,10.52,769.04M,57.49,1.76,5.61,827.43M,658.57M


<h2>Reformatting large sums (Billion, million, thousand) and removing (%,) values</h2>

In [155]:
import re

def cleaning(x):
    x = re.sub("[,]", "", x)
    if x[-1] == 'T':
        x = round(float(x[:-1])*1000,2)
    elif x[-1] == 'B':
        x = round(float(x[:-1]),2)
    elif x[-1] == 'M':
        x = round(float(x[:-1])*0.001,2)
    elif x[-1] == 'k':
        x = round(float(x[:-1])*0.000001,2)
    elif x[-1] == '%':
        x = round(float(x[:-1]),0)             
    elif x == "N/A":
        x = 0
    return x

cleaned_df = t_all_stats_df.iloc[:,1:].applymap(cleaning)
cleaned_df.insert(0, 'Ticker', tickers)
cleaned_df.head()

Metrics,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy) (%),Total Cash (mrq) (B),Total Debt (mrq) (B),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B)
0,TSLA,395.82,380.59,38.77,24.69,1.10,5.78,9.93,5.08,23.48,...,3.23,104.0,21.11,6.68,5.87,14.28,1.46,12.62,16.03,6.55
1,TTD,22.81,21.75,0,40.16,2.33,15.39,11.91,14.67,572.96,...,0.08,-73.0,1.32,2.7,0.27,13.94,1.91,3.91,0.54,0.54
2,NVDA,405.92,404.53,70.22,38.31,3.92,14.58,19.01,14.16,54.34,...,3.73,-72.0,13.14,5.34,11.90,55.76,3.39,8.65,6.43,5.15
3,GOOG,1140.00,1050.00,17.55,16.50,1.24,4.16,4.50,3.73,11.17,...,4.92,-26.0,116.26,8.98,29.34,11.57,2.52,19.55,92.81,48.26
4,CRWD,25.16,23.46,0,51.02,1.25,12.23,19.15,11.53,-711.89,...,-0.79,0.0,2.47,10.52,0.77,57.49,1.76,5.61,0.83,0.66


<h2>Handling the stock split factor column</h2>

In [186]:
print('Before')
display(cleaned_df.iloc[:,[0,37]].head(10))

print('After')
def stocksplits(x):
    if type(x)==str:
        x = x.split(':')
        return round(int(x[0])/ int(x[1]),2)
    else:
        return x
    
cleaned_df['Last Split Factor (x:1)'] = cleaned_df['Last Split Factor (x:1)'].apply(stocksplits)
display(cleaned_df.iloc[:,[0,37]].head(11))

Before


Metrics,Ticker,Last Split Factor (x:1)
0,TSLA,3:1
1,TTD,10:1
2,NVDA,4:1
3,GOOG,20:1
4,CRWD,0
5,AMZN,20:1
6,IDXX,2:1
7,MELI,0
8,ABNB,0
9,ZS,0


After


Metrics,Ticker,Last Split Factor (x:1)
0,TSLA,3.00
1,TTD,10.00
2,NVDA,4.00
3,GOOG,20.00
4,CRWD,0.00
5,AMZN,20.00
6,IDXX,2.00
7,MELI,0.00
8,ABNB,0.00
9,ZS,0.00


<h2>Converting dates to datetime format at the end of the table</h2>

In [184]:
dates = cleaned_df.iloc[:,[35, 36, 38, 39, 40]]
def date_conversion(x):
    if x == 0:
        return np.nan
    else:
        x = str(x)
        return datetime.strptime(x, '%b %d %Y').date()
    
    
dates = dates.applymap(date_conversion)
dates = dates.replace({0:np.nan})
#dates = pd.to_datetime(dates, errors = 'ignore')
dates.head()


Metrics,Dividend Date,Ex-Dividend Date,Last Split Date,Fiscal Year Ends,Most Recent Quarter (mrq)
0,NaN,NaN,2022-08-24,2021-12-30,2022-09-29
1,NaN,NaN,2021-06-16,2021-12-30,2022-09-29
2,2022-12-21,2022-11-29,2021-07-19,2022-01-29,2022-10-29
3,NaN,NaN,2022-07-17,2021-12-30,2022-09-29
4,NaN,NaN,NaN,2022-01-30,2022-10-30


In [188]:
final_df = cleaned_df.drop(cleaned_df.columns[[0, 35, 36, 38, 39, 40]], axis = 1).astype('float64')
final_df = pd.concat([final_df, dates], axis = 1)
final_df.insert(0, 'Ticker', tickers)
final_df.head()

Metrics,Ticker,Market Cap (B),Enterprise Value (B),Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm) (B),Levered Free Cash Flow (ttm) (B),Dividend Date,Ex-Dividend Date,Last Split Date,Fiscal Year Ends,Most Recent Quarter (mrq)
0,TSLA,395.82,380.59,38.77,24.69,1.10,5.78,9.93,5.08,23.48,...,14.28,1.46,12.62,16.03,6.55,NaN,NaN,2022-08-24,2021-12-30,2022-09-29
1,TTD,22.81,21.75,0.00,40.16,2.33,15.39,11.91,14.67,572.96,...,13.94,1.91,3.91,0.54,0.54,NaN,NaN,2021-06-16,2021-12-30,2022-09-29
2,NVDA,405.92,404.53,70.22,38.31,3.92,14.58,19.01,14.16,54.34,...,55.76,3.39,8.65,6.43,5.15,2022-12-21,2022-11-29,2021-07-19,2022-01-29,2022-10-29
3,GOOG,1140.00,1050.00,17.55,16.50,1.24,4.16,4.50,3.73,11.17,...,11.57,2.52,19.55,92.81,48.26,NaN,NaN,2022-07-17,2021-12-30,2022-09-29
4,CRWD,25.16,23.46,0.00,51.02,1.25,12.23,19.15,11.53,-711.89,...,57.49,1.76,5.61,0.83,0.66,NaN,NaN,NaN,2022-01-30,2022-10-30


In [190]:
current_datetime = date.today().isoformat()
final_df.to_csv(f'data/portfolio_{current_datetime}.csv')
print(f'Saved to file: portfolio_{current_datetime}.csv')

Saved to file: portfolio_2022-12-24.csv
